In [1]:
#Andrew Weis
#PPHA 30531 Data Skills for Public Policy
#PSet5
#26 November 2019

#Import packages
import pandas as pd
import bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
#File paths
PATH='/Users/andrewweis/Documents/GitHub/DataSkills/'
HW_FOLDER='assignment-5-awastro55'
DATA_FOLDER='chicago-police-data/data'
COMPLAINTS='/unified_data/complaints/'
DISCIPLINE='/context_data/'

In [3]:
#Generate the csv paths.
base_csv='complaints-{}_2000-2016_2016-11.csv'
datasets=['accused','victims','investigators','witnesses','complainants']
discipline_file='discipline_penalty_codes.csv'

def csv_gen(sets,base):
    return base.format(sets)

complaints_files=[csv_gen(item, base_csv) for item in datasets]

In [4]:
def read_data(csv):
    complaints_data=[]
    for item in csv:
        complaints_data.append(pd.read_csv(PATH+DATA_FOLDER+COMPLAINTS+item))
        
    return complaints_data

In [5]:
complaints_data=read_data(complaints_files)
discipline=pd.read_csv(PATH+DATA_FOLDER+DISCIPLINE+discipline_file)
#Now we have our data frames.  

In [6]:
#Find data types
def check(df):
    return df.dtypes

check(discipline)

CODE             int64
ACTION_TAKEN    object
NOTES           object
dtype: object

In [7]:
check(complaints_data[0])

row_id                                       int64
complaints-accused_2000-2016_2016-11_ID    float64
cr_id                                        int64
complaint_category                          object
recommended_discipline                     float64
final_discipline                           float64
recommended_finding                         object
final_finding                               object
UID                                          int64
old_UID                                      int64
link_UID                                   float64
dtype: object

In [8]:
check(complaints_data[1])

cr_id       int64
gender     object
age       float64
race       object
dtype: object

In [9]:
check(complaints_data[2])

row_id                                             int64
complaints-investigators_2000-2016_2016-11_ID      int64
cr_id                                              int64
first_name                                        object
last_name                                         object
middle_initial                                    object
suffix_name                                       object
appointed_date                                    object
current_star                                     float64
current_rank                                      object
current_unit                                     float64
UID                                              float64
old_UID                                          float64
link_UID                                         float64
dtype: object

In [10]:
check(complaints_data[3])

row_id                                         int64
complaints-witnesses_2000-2016_2016-11_ID    float64
cr_id                                          int64
UID                                          float64
link_UID                                     float64
dtype: object

In [11]:
check(complaints_data[4])

cr_id       int64
gender     object
age       float64
race       object
dtype: object

In [14]:
complaints_witnesses=complaints_data[3]

In [8]:
officer=PATH+DATA_FOLDER+COMPLAINTS+'officer-filed-complaints__2017-09.csv'
officer_df=pd.read_csv(officer)
officer

'/Users/andrewweis/Documents/GitHub/DataSkills/chicago-police-data/data/unified_data/complaints/officer-filed-complaints__2017-09.csv'

In [24]:
check(officer_df)

cr_id    int64
dtype: object

In [12]:
def new_column(df, original, new):
    df[new]=df[original]
    return df

In [13]:
#Copy columns in discipline frame to get ready for merging.  
discipline=new_column(discipline,'CODE','recommended_discipline')
discipline=new_column(discipline,'CODE','final_discipline')
discipline=new_column(discipline,'ACTION_TAKEN','recommended_action')
discipline=new_column(discipline,'ACTION_TAKEN','final_action')
#discipline

In [14]:
#separate the data frames
complaints_accused=complaints_data[0]
complaints_victims=complaints_data[1]
complaints_investigators=complaints_data[2]
complaints_complainants=complaints_data[4]

In [15]:
def merging():
    accrec=pd.merge(complaints_accused,discipline[['recommended_discipline','recommended_action']],
                    how='left',on='recommended_discipline')
    accnew=pd.merge(accrec,discipline[['final_discipline','final_action']], 
                how='left',on='final_discipline')
    accvic=pd.merge(accnew, complaints_data[1], how='outer', on='cr_id')
    acccom=pd.merge(accvic, complaints_data[4],how='outer', on='cr_id',
                   suffixes=('_vic', '_com'))
    accfinal=pd.merge(acccom,complaints_data[2][['cr_id', 'row_id',
                                             'complaints-investigators_2000-2016_2016-11_ID',
                                             'first_name','last_name',
                                             'middle_initial',
                                             'suffix_name',
                                             'appointed_date',
                                             'current_star',
                                             'current_rank',
                                             'current_unit']],how='outer',on='cr_id',
                     suffixes=('_acc','_inv'))
    return accfinal

In [16]:
accfinal=merging()
accfinal

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [17]:
final_complaint=accfinal.drop_duplicates()
final_complaint

,row_id_acc,complaints-accused_2000-2016_2016-11_ID,cr_id,complaint_category,recommended_discipline,final_discipline,recommended_finding,final_finding,UID,old_UID,...,row_id_inv,complaints-investigators_2000-2016_2016-11_ID,first_name,last_name,middle_initial,suffix_name,appointed_date,current_star,current_rank,current_unit
0,1.0,15646.0,258996,01A-USE OF PROFANITY,600.0,600.0,NS,NS,130754.0,1421.0,...,1.0,1.0,STEVEN,SCHWIEGER,NaN,NaN,1986-06-16,NaN,LIEUTENANT OF POLICE,13.0
1,2.0,8602.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,117163.0,3556.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
2,3.0,6951.0,258997,01A-USE OF PROFANITY,600.0,600.0,UN,UN,113991.0,14292.0,...,2.0,2.0,MICHAEL,MULLIGAN,NaN,JR,1970-12-14,NaN,SERGEANT OF POLICE,8.0
3,4.0,13745.0,258998,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,600.0,600.0,EX,EX,127168.0,26608.0,...,3.0,3.0,MAUREEN,MC MAHON,NaN,NaN,1985-07-01,NaN,LIEUTENANT OF POLICE,608.0
4,5.0,918.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,101654.0,17539.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
5,6.0,10477.0,259001,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,600.0,600.0,UN,UN,120493.0,13708.0,...,6.0,6.0,CHARLES,POPIELARZ,NaN,NaN,1982-06-14,NaN,SERGEANT OF POLICE,1.0
6,7.0,2403.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,104455.0,3055.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
7,8.0,8339.0,259002,05A-ARRESTEE - DURING ARREST,600.0,600.0,NS,NS,116784.0,19347.0,...,7.0,7.0,MONIKA,NORWOOD,NaN,NaN,1993-05-01,NaN,INVESTIGATOR 2 IPRA,113.0
8,9.0,14036.0,259005,07A-MISCONDUCT DURING ISSUANCE OF CITATION,600.0,600.0,NS,NS,127595.0,28345.0,...,9.0,9.0,BEATRICE,O'DONNELL,NaN,NaN,1987-04-13,NaN,CHIEF,120.0
9,10.0,5741.0,259008,09G-ABUSE OF AUTHORITY,600.0,600.0,UN,UN,111265.0,2829.0,...,12.0,12.0,THOMAS,RENNIE,NaN,NaN,1970-06-15,NaN,CAPTAIN OF POLICE,1.0


In [18]:
by_crid=final_complaint.groupby(['complaint_category']).size().reset_index().rename(columns={0:"number"})
by_crid

,complaint_category,number
0,01A-USE OF PROFANITY,3373
1,"01B-RACIAL/ETHNIC, ETC.",1536
2,01C-MISCELLANEOUS,864
3,02A-INTOXICATED ON DUTY,73
4,02B-INTOXICATED OFF DUTY,131
5,02C-D.U.I. - ON DUTY,10
6,02D-D.U.I. - OFF DUTY,171
7,02E-POSSESSION/DRINKING ALCOHOL - ON DUTY,24
8,02G-MISCELLANEOUS,15
9,03A-FIRST AMENDMENT,66


In [20]:
by_race=final_complaint.groupby(['race_com', 'final_finding']).nunique('cr_id')
check(final_complaint)

row_id_acc                                       float64
complaints-accused_2000-2016_2016-11_ID          float64
cr_id                                              int64
complaint_category                                object
recommended_discipline                           float64
final_discipline                                 float64
recommended_finding                               object
final_finding                                     object
UID                                              float64
old_UID                                          float64
link_UID                                         float64
recommended_action                                object
final_action                                      object
gender_vic                                        object
age_vic                                          float64
race_vic                                          object
gender_com                                        object
age_com                        

In [21]:
check(by_crid)

complaint_category    object
number                 int64
dtype: object

In [22]:
by_race

row_id_acc  \
race_com                       final_finding               
ASIAN/PACIFIC ISLANDER         EX                     25   
                               NAF                   207   
                               NS                     61   
                               SU                     45   
                               UN                     47   
BLACK                          EX                   1531   
                               NAF                 18521   
                               NC                      1   
                               NS                   7711   
                               SU                    980   
                               UN                   4275   
HISPANIC                       EX                    236   
                               NAF                  2651   
                               NS                   1418   
                               SU                    434   
                               UN                    729   
NATIVE AMERICAN/ALASKAN NATIVE NAF                    21   
                               NS                      8   
                               SU                      2   
                               UN                      4   
WHITE                          EX                    458   
                               NAF                  3758   
                               NS                   1826   
                               SU                   1990   
                               UN                   1276   

                                              complaints-accused_2000-2016_2016-11_ID  \
race_com                       final_finding                                            
ASIAN/PACIFIC ISLANDER         EX                                                  25   
                               NAF                                                203   
                               NS                                                  60   
                               SU                                                  45   
                               UN                                                  47   
BLACK                          EX                                                1332   
                               NAF                                               6991   
                               NC                                                   1   
                               NS                                                4246   
                               SU                                                 881   
                               UN                                                3124   
HISPANIC                       EX                                                 231   
                               NAF                                               1975   
                               NS                                                1187   
                               SU                                                 403   
                               UN                                                 676   
NATIVE AMERICAN/ALASKAN NATIVE NAF                                                 21   
                               NS                                                   8   
                               SU                                                   2   
                               UN                                                   4   
WHITE                          EX                                                 448   
                               NAF                                               2889   
                               NS                                                1579   
                               SU                                                1683   
                               UN                                                1179   

                     

In [23]:
check(by_race)

row_id_acc                                       int64
complaints-accused_2000-2016_2016-11_ID          int64
cr_id                                            int64
complaint_category                               int64
recommended_discipline                           int64
final_discipline                                 int64
recommended_finding                              int64
final_finding                                    int64
UID                                              int64
old_UID                                          int64
link_UID                                         int64
recommended_action                               int64
final_action                                     int64
gender_vic                                       int64
age_vic                                          int64
race_vic                                         int64
gender_com                                       int64
age_com                                          int64
race_com  

In [19]:
def complaints_plot():
    plot = figure(title='Complaints by category', x_axis_label='Type', y_axis_label='Number',
             x_range=by_crid['complaint_category'])
    plot.vbar(x=by_crid['complaint_category'], top=by_crid["number"], width=0.2)
    show(plot)

complaints_plot()

In [38]:
by_crid['number'][:3]

0    3373
1    1536
2     864
Name: number, dtype: int64

In [ ]:
#I haven't been able to get the plots to work.  I wanted to make plots based on race, but
#grouping by race won't work, even though it's the same line of code as in the prior assignment.
#Just copied and pasted.  I also still couldn't figure out how to generalize the merging
#function.  In the spirit of observing the deadline, I'll submit my failed attempts. To
#improve this, I would generalize the merging function, create plotting function, and plot:
#number of complaints filed by people of each race.  Number of complaints sustained by people
#of each race.  Complaints sustained by category.  I will make further attempts to fix this
#code and may submit a correction if/when I achieve one. 

In [25]:
def race_plot():
    plot=figure(title='Complaint Findings by Race', x_axis_label='Complaint Outcome', 
                y_axis_label='Number', x_range=by_race['final_finding'])
    plot.vbar(x=by_race['final_finding'], top=by_race['row_id_acc'], width=0.4)
    show(plot)
    
race_plot()

ValueError: Unrecognized range input: '[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]'

In [24]:
by_race_only=final_complaint.groupby(['race_com']).size().reset_index().rename(columns={0:"number"})#.nunique('cr_id')
by_race_only

,race_com,number
0,ASIAN/PACIFIC ISLANDER,791
1,BLACK,67603
2,HISPANIC,11983
3,NATIVE AMERICAN/ALASKAN NATIVE,65
4,WHITE,20376


In [20]:
by_final_finding=final_complaint.groupby(['final_finding']).size().reset_index().rename(columns={0:"number"})#.nunique('cr_id')
by_final_finding

,final_finding,number
0,DIS,12
1,EX,9899
2,NAF,46575
3,NC,60
4,NS,48104
5,SU,10457
6,UN,35429


In [22]:
def findings_plot():
    plot = figure(title='Findings', x_axis_label='Finding', y_axis_label='Number',
             x_range=by_final_finding['final_finding'])
    plot.vbar(x=by_final_finding['final_finding'], top=by_final_finding["number"], width=0.2)
    show(plot)

    
findings_plot()

In [26]:
def race_only_plot():
    plot = figure(title='Race', x_axis_label='Race', y_axis_label='Number',
             x_range=by_race_only['race_com'])
    plot.vbar(x=by_race_only['race_com'], top=by_race_only["number"], width=0.2)
    show(plot)

    
race_only_plot()